##### genetic algorithm

In [18]:
import random as rd
from ctypes import *
from bitsets import bitset
from bitarray import bitarray
import bitstring
import numpy as np

In [33]:
class GeneticAlgorithm():
    
    def __init__(self, x_min, x_max, z_min, z_max, m, pop_c, prob):
        self.x_min = x_min
        self.x_max = x_max
        self.z_min = z_min
        self.z_max = z_max
        self.mutation_probability = m  #вероятность мутации
        self.population_count = pop_c  #население
        self.cross_probability = prob  #вероятность скрещивания
        
        step_x = (self.x_max - self.x_min) / self.population_count         #шаг по оси х
        step_z = (self.z_max - self.z_min) / self.population_count         #шаг по оси y
         
        self.data = []       #храним пары [x, y]
        self.res  = []       #храним результаты из функции

        for idx in range(0, self.population_count):
            self.data.append([self.x_min + step_x * idx, self.z_min + step_z * idx]);
            self.res.append(self.function(self.data[idx][0], self.data[idx][1]));
    
    def tournamentSelection(self):
        buff = []
        rb = []

        while len(buff) != self.population_count:
        
            t1 = rd.randint(0, 10000) % len(self.data)
            t2 = rd.randint(0, 10000) % len(self.data)

            if self.res[t1] > self.res[t2]:
                buff.append(self.data[t1])
                rb.append(self.res[t1])

                self.data[t1], self.data[-1] = self.data[-1], self.data[t1] 
                self.res[t1],  self.res[-1]  = self.res[-1],  self.res[t1]
                self.data.pop()
                self.res.pop()
            else:
                buff.append(self.data[t2])
                rb.append(self.res[t2])

                self.data[t2], self.data[-1] = self.data[-1], self.data[t2] 
                self.res[t2],  self.res[-1]  = self.res[-1],  self.res[t2]
                self.data.pop()
                self.res.pop()

        self.data = buff
        self.res  = rb
        
        
    def function(self, x, z):
        #sin(sqrt(x*x+y*y))
        return float(np.sqrt(np.abs(np.sin(x*x+z*z))))
        #return float(-(x * x + z * z))

    def kill(self, c1, c2):
        if c1[0] > self.x_max or c1[0] < self.x_min or c1[1] > self.z_max or c1[1] < self.z_min or c2[0] > self.x_max or c2[0] < self.x_min or c2[1] > self.z_max or c2[1] < self.z_min:
            return True
        else:
            return False
    
    
    def searchBest(self):
        color = [False] * len(self.data)       #закрашиваем тех, кого скрестили
                                               #используем случайный выбор для кроссинговера
        for k in range(0, self.population_count):
            j = rd.randint(0, 100000) % self.population_count
            i = rd.randint(0, 100000) % self.population_count

            if color[i] == False:
                b1 = self.data[j]
                b2 = self.data[i]
                self.crossingOver(b1, b2)

                if (self.kill(b1, b2) == False):
                    self.data.append(b1)
                    self.data.append(b2)
                    self.res.append(self.function(b1[0], b1[1]))
                    self.res.append(self.function(b2[0], b2[1]))
            
                color[i] = True
                color[j] = True
        #после скрещивания надо провести отбор
        self.tournamentSelection()
    
    def mutation(self, c):
        class actual_data( Union ):
            _fields_ = [( "in_", c_float ), ( 'out_', c_ulong )]
        uni = actual_data()

        uni.in_ = c
        f1 = bitstring.BitArray(float = uni.out_, length = 32) 
        bi = f1.bin
        a = bitarray(bi)

        pred = rd.randint(0, 100000) % 1000 / 1000;

        if pred < self.mutation_probability:
            idx = rd.randint(0, 100000) % 31
            a[idx] = 0 if (a[idx] == 1) else 1
            uni.out_ = int(a.to01(), 2)

        return uni.in_
    
    
    def crossingOver(self, cop_1, cop_2):
        for i in 0, 1:
            class actual_data( Union ):
                _fields_ = [( "in_", c_float ), ( 'out_', c_ulong )]
            uni_1 = actual_data()
            uni_2 = actual_data()

#             print(uni_1.in_, uni_1.out_)
            
            uni_1.in_ = cop_1[i]
            uni_2.in_ = cop_2[i]
           # print("до", uni_1.in_, uni_2.in_, uni_1.out_, uni_2.out_)

            f1 = bitstring.BitArray(float = uni_1.out_, length = 32) 
            bi1 = f1.bin
            bits_1 = bitarray(bi1)
            bits_1_buff = bitarray(bi1)
            #print("bits1", bits_1, bits_1_buff)
            
            f2 = bitstring.BitArray(float = uni_2.out_, length = 32)
            bi2 = f2.bin
            bits_2 = bitarray(bi2)
            bits_2_buff = bitarray(bi2)
            #print("bits2", bits_2, bits_2_buff)

            #print("buff do   ", bits_1_buff, bits_2_buff)

            flag = rd.randint(0, 100000) % 31
            for j in range(flag, 0, -1):
                bits_2_buff[j] = bits_1[j]
                bits_1_buff[j] = bits_2[j]
            
            
           # print("buff после", bits_1_buff, bits_2_buff)
            uni_1.out_ = int(bits_1_buff.to01(), 2)
            uni_2.out_ = int(bits_2_buff.to01(), 2)
            
            #print("out после", uni_1.out_, uni_2.out_)

            
            #print("in после", uni_1.in_, uni_2.in_)
            
            cop_1[i] = self.mutation(float(uni_1.in_))
            cop_2[i] = self.mutation(float(uni_2.in_))
            #print("после", cop_1[i], cop_2[i])
    
    def result(self):
        x = 0
        y = 0
        result = -2147483648

        for i in range(0, len(self.data)):
            if self.res[i] > result:
                x = self.data[i][0]
                y = self.data[i][1]
                result = self.res[i]

#         return {x, y, result}
        return {'x': x, 'y': y, "result": result}


In [37]:
test = GeneticAlgorithm(-100, 100, -100, 100, 0, 1000, 1)

In [38]:
for i in range(0, 100):
    test.searchBest()

In [39]:
print(test.result())

{'x': 1318926976.0, 'y': 1318926976.0, 'result': 0.9999621210737044}
